# Kamodo Model/Data Comparison Notebook

## CCMC RoR extraction model/data comparison
#### Start with load satellite extractions from model output

In [ ]:
# Import some libraries
from kamodo import Kamodo
from kamodo_ccmc.readers.hapi import HAPI
from kamodo_ccmc.readers.sat_extractions import SATEXTRACT,ror_show_info
from kamodo_ccmc.flythrough.wrapper_output import Functionalize_TimeSeries

In [ ]:
# Look up information on a specific run
runID     = '13819'
ror_show_info(runID)

In [ ]:
# Load this run data in specified coordinates for given satellite
coord     = 'GSE'     # Can choose from GSE, GSM, SM
satellite = 'GOES-12'

ror = SATEXTRACT(runID, coord, satellite, debug=0)
ror

In [ ]:
# Plot magnetic field
ror.get_plot(type="1Dvar", var="B")

In [ ]:
# Plot 3D magnetic field with vectors, grouped by day with slider
ror.get_plot(type="3Dvar", var="B_x", groupby="day", quiver=True, quiverskip=10)

#### Read the same satellite and dates from CDAWeb

In [ ]:
server = 'https://cdaweb.gsfc.nasa.gov/hapi'
dataset = 'GOES12_K0_MAG'
parameters = 'B_GSE_c'
start      = ror.start
stop       = ror.stop

hapi2 = HAPI(server, dataset, parameters, start, stop)

In [ ]:
# Plot the magnetic field as observed by GOES 12
hapi2.get_plot(type="1Dvar", var="B_GSE_c")

#### Model / Data Comparison

In [ ]:
# Add variable from one Kamodo object to another to plot together
ror['B_z__GSE_GOES12']=hapi2['B_z__GSE_c']

In [ ]:
# Interpolate model data B_z onto same time series as observed data
interpB_z = ror.B_z__GSE(hapi2.dtarray)

# Compute difference of two identical length arrays
deltaB_z = hapi2.variables['B_GSE_c']['data'][:,2] - interpB_z

# Add the new time series back into the Kamodo object for further analysis/plotting
ror = Functionalize_TimeSeries(hapi2.tsarray, 'DIFFERENCE', 'nT', deltaB_z, kamodo_object=ror)

In [ ]:
# Now we can plot model and data on the same figure with the difference
ror.plot('B_z__GSE','B_z__GSE_GOES12','DIFFERENCE')